In [25]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community faiss-cpu

In [26]:
# Enable tracing
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Call LLM Model

In [27]:
# Add phi3:3.8b LLM model
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="phi3:3.8b",
)

In [28]:
# Test LLM model is working
response_message = model.invoke("What is the capital of France?")

print(response_message.content)

The capital of France is Paris. It'ran city with a rich history that dates back to its founding in the 3rd century BC, known for iconic landmarks such as the Eiffel Tower and the Louvre Museum. The Île de la Cité, where Paris was founded, lies within two rivers: the Seine and the Bièvre which once passed through but is now largely underground due to urban development in the 19th century. Paris serves not only as a political center of France with its status as the seat of government institutions like the Elysée Palace where the President lives, but also as an economic powerhouse and one of Europe's most vibrant cultural hubs, home to countless museums, galleries, cafes, and boulevards that reflect a unique Parisian lifestyle.


In [29]:
import pandas as pd
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import random
import sqlite3

class QuizChatbot:
    def __init__(self, db_path: str):
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
            
        # Execute a query to retrieve the data
        query = "SELECT * FROM jeopardy"
        self.df = pd.read_sql_query(query, conn)
            
        # Close the connection
        conn.close()
        
        self.categories = self.df['Category'].unique().tolist()
            
        # Initialize the Ollama model
        self.llm = ChatOllama(
            model="phi3:3.8b",
            temperature=0
        )
            
        # Initialize state variables
        self.current_question = None
        self.current_answer = None

    def get_categories(self) -> str:
        """Return formatted list of categories"""
        return "\n".join([f"{i+1}. {cat}" for i, cat in enumerate(self.categories)])
    
    def select_question(self, category: str) -> tuple:
        """Select a random question from the specified category"""
        category_questions = self.df[self.df['Category'] == category]
        if len(category_questions) == 0:
            return None, None
        
        question_row = category_questions.sample(n=1).iloc[0]
        return question_row['Question'], question_row['Answer']
    
    def verify_answer(self, user_answer: str, correct_answer: str) -> bool:
        """Use the LLM to verify if the answer is correct"""
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are an answer verification expert. 
            Compare the user's answer with the correct answer and determine if they are semantically equivalent.
            Respond with only 'True' if correct or 'False' if incorrect."""),
            HumanMessage(content=f"""
            Correct answer: {correct_answer}
            User's answer: {user_answer}
            Are these answers semantically equivalent?
            """)
        ])
        
        response = self.llm.invoke(prompt.format_messages())
        return 'true' in response.content.lower()
    
    def run(self):
        """Main interaction loop"""
        print("Welcome to the Quiz Chatbot!")
        
        while True:
            print("\nAvailable categories:")
            print(self.get_categories())
            
            # Get category selection
            selection = input("\nPlease select a category number (or 'quit' to exit): ")
            
            if selection.lower() == 'quit':
                print("Thank you for playing!")
                break
                
            try:
                category_index = int(selection) - 1
                if 0 <= category_index < len(self.categories):
                    selected_category = self.categories[category_index]
                    
                    # Get random question from category
                    question, answer = self.select_question(selected_category)
                    if question is None:
                        print("Error: No questions available in this category.")
                        continue
                    
                    # Ask question and get user's answer
                    print(f"\nQuestion: {question}")
                    user_answer = input("Your answer: ")
                    
                    # Verify answer
                    is_correct = self.verify_answer(user_answer, answer)
                    
                    # Provide feedback
                    if is_correct:
                        print("Correct! Well done!")
                    else:
                        print(f"Incorrect. The correct answer was: {answer}")
                else:
                    print("Invalid category number. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

# Usage
db_path = "../data/prod/trivia_qa.db"
if __name__ == "__main__":
    chatbot = QuizChatbot(db_path)
    chatbot.run()


Welcome to the Quiz Chatbot!

Available categories:
1. SCIENCE
2. LITERATURE
3. AMERICAN HISTORY

Question: The wife of this Sinclair Lewis research scientist dies during an epidemic in the West Indies
Incorrect. The correct answer was: Arrowsmith

Available categories:
1. SCIENCE
2. LITERATURE
3. AMERICAN HISTORY

Question: Part of a corkscrew is this type of simple machine reworked into a spiral form
Incorrect. The correct answer was: an inclined plane (ramp later ruled acceptable)

Available categories:
1. SCIENCE
2. LITERATURE
3. AMERICAN HISTORY

Question: (<a href="http://www.j-archive.com/media/2008-07-04_DJ_10.jpg" target="_blank">Jimmy of the Clue Crew explains a chemical reaction</a>) A chemical reaction between food coloring and bleach, which contains <a href="http://www.j-archive.com/media/2008-07-04_DJ_10a.jpg" target="_blank">this</a> chemical, NaClO3, turns the color liquid into a colorless compound
Incorrect. The correct answer was: sodium chlorate

Available categories